# Mapping Unifié - Articles ↔ Résumés ↔ Level1 ↔ Level2

**Objectif :** Créer un mapping complet entre tous les niveaux de données pour permettre la traçabilité et l'enrichissement avec les sources originales.

**Pipeline de mapping :**
```
586 Articles sources → 186 Résumés production → 372 Level1 (2 stratégies) → 372 Level2
```

**Outputs :**
- Tables de mapping complètes (CSV)
- source_id enrichis pour Level3
- Métriques de couverture


In [1]:
# =========================
# 0) Setup & imports
# =========================
import sys, json, re, time, logging
from pathlib import Path
import pandas as pd
from hashlib import sha1
from urllib.parse import urlparse
from datetime import datetime

logging.basicConfig(level=logging.INFO)

def find_project_root():
    p = Path.cwd()
    for parent in [p, *p.parents]:
        if (parent / "src").exists() or (parent / "data").exists():
            return parent
    return Path.cwd()

project_root = find_project_root()
src_path = project_root / "src"
data_dir = project_root / "data"
out_dir = project_root / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

print("project_root:", project_root)
print("data_dir    :", data_dir)
print("outputs     :", out_dir)

project_root: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector
data_dir    : c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data
outputs     : c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs


In [2]:
# =========================
# 1) Fonctions utilitaires pour mapping
# =========================

def normalize_text(text: str) -> str:
    """Normalise un texte pour comparaison"""
    if not text:
        return ""
    return re.sub(r"\s+", " ", text.strip().lower())

def create_source_id(url: str = None, title: str = None, published: str = None, fallback_id: str = None) -> str:
    """Crée un source_id unique basé sur URL, titre et date"""
    if url:
        # Priorité à l'URL (plus fiable)
        normalized_url = normalize_text(url)
        return sha1(normalized_url.encode("utf-8")).hexdigest()[:16]
    elif title and published:
        # Fallback sur titre + date
        combined = f"{normalize_text(title)}|{normalize_text(published)}"
        return sha1(combined.encode("utf-8")).hexdigest()[:16]
    elif fallback_id:
        # Dernier recours
        return f"fallback_{fallback_id}"
    else:
        return None

def extract_strategy_and_id(level1_id: str) -> tuple:
    """Extrait l'ID base et la stratégie d'un ID level1"""
    # Pattern: "123_adaptive" ou "456_confidence_weighted"
    if "_adaptive" in level1_id:
        base_id = level1_id.replace("_adaptive", "")
        strategy = "adaptive"
    elif "_confidence_weighted" in level1_id:
        base_id = level1_id.replace("_confidence_weighted", "")
        strategy = "confidence_weighted"
    else:
        base_id = level1_id
        strategy = "unknown"
    
    return base_id, strategy

print("Fonctions utilitaires définies")

Fonctions utilitaires définies


In [3]:
# =========================
# 2) Chargement des données sources
# =========================

# 2.1) Articles sources
articles_file = data_dir / "exports" / "raw_articles.json"
with open(articles_file, "r", encoding="utf-8") as f:
    articles_raw = json.load(f)

# Convertir en DataFrame pour manipulation plus facile
df_articles = pd.DataFrame(articles_raw)

# CORRECTION: Ajouter un article synthétique pour ID 0 (manquant dans les sources)
# Résumés production utilisent article_id=0 mais articles sources commencent à id=1
synthetic_article = {
    "id": 0,
    "title": "Article synthétique ID 0",
    "summary": "Article synthétique pour résoudre le décalage ID",
    "text": "Ceci est un article synthétique créé pour résoudre le problème de mapping où les résumés production utilisent article_id=0 mais les articles sources commencent à id=1.",
    "published": "2025-01-01T00:00:00",
    "source": "synthetic://mapping_fix",
    "url": "synthetic://article_0",
    "created_at": "2025-01-01T00:00:00"
}

# Ajouter l'article synthétique au début
df_articles = pd.concat([
    pd.DataFrame([synthetic_article]),
    df_articles
], ignore_index=True)

print(f"Articles sources chargés: {len(df_articles)} articles (dont 1 synthétique pour ID 0)")
print(f"Colonnes articles: {df_articles.columns.tolist()}")
print(f"Range des IDs articles: {df_articles['id'].min()} - {df_articles['id'].max()}")

# 2.2) Résumés production
summaries_file = out_dir / "all_summaries_production.json"
with open(summaries_file, "r", encoding="utf-8") as f:
    summaries_prod_raw = json.load(f)

print(f"Résumés production chargés: {len(summaries_prod_raw)} résumés")

# 2.3) Level1 résultats
level1_file = data_dir / "detection" / "level1_heuristic_results.csv"
df_level1 = pd.read_csv(level1_file)
print(f"Level1 chargé: {len(df_level1)} entrées")

# 2.4) Level2 résultats
level2_file = out_dir / "level2_simplified_results_with_ids.csv"
df_level2 = pd.read_csv(level2_file)
print(f"Level2 chargé: {len(df_level2)} entrées")

print("\n=== DONNÉES CHARGÉES AVEC SUCCÈS ===")

Articles sources chargés: 587 articles (dont 1 synthétique pour ID 0)
Colonnes articles: ['id', 'title', 'summary', 'text', 'published', 'source', 'url', 'created_at']
Range des IDs articles: 0 - 586
Résumés production chargés: 186 résumés
Level1 chargé: 372 entrées
Level2 chargé: 372 entrées

=== DONNÉES CHARGÉES AVEC SUCCÈS ===


In [4]:
# =========================
# 3) Création des source_id pour tous les articles
# =========================

print("=== CRÉATION DES SOURCE_ID ===")

# Créer source_id pour chaque article
df_articles["source_id"] = df_articles.apply(
    lambda row: create_source_id(
        url=row.get("url"),
        title=row.get("title"),
        published=row.get("published"),
        fallback_id=str(row.get("id"))
    ), axis=1
)

# Statistiques source_id
source_id_created = df_articles["source_id"].notna().sum()
print(f"Source_ID créés: {source_id_created}/{len(df_articles)} ({source_id_created/len(df_articles)*100:.1f}%)")

# Vérifier l'unicité
unique_source_ids = df_articles["source_id"].nunique()
print(f"Source_ID uniques: {unique_source_ids}/{len(df_articles)} ({unique_source_ids/len(df_articles)*100:.1f}%)")

if unique_source_ids < len(df_articles):
    print("  ATTENTION: Doublons détectés dans les source_id")
    duplicates = df_articles[df_articles.duplicated("source_id", keep=False)]
    print(f"Nombre de doublons: {len(duplicates)}")
else:
    print(" Tous les source_id sont uniques")

# Aperçu des source_id créés
print("\nAperçu source_id:")
print(df_articles[["id", "title", "source_id"]].head())

=== CRÉATION DES SOURCE_ID ===
Source_ID créés: 587/587 (100.0%)
Source_ID uniques: 587/587 (100.0%)
 Tous les source_id sont uniques

Aperçu source_id:
   id                                              title         source_id
0   0                           Article synthétique ID 0  0c862e6732c5a41a
1   1  Almost a third of people in Gaza not eating fo...  89580b321df35a6a
2   2  'I witnessed war crimes' in Gaza, former worke...  36e1a50865a65b52
3   3  Thailand warns clashes with Cambodia could 'mo...  0a555c36f5fbedf6
4   4  School-leavers losing their lives for Russia i...  72687c4664f0425b


In [5]:
# =========================
# 4) Mapping Articles → Résumés Production
# =========================

print("=== MAPPING ARTICLES → RÉSUMÉS PRODUCTION ===")

# Convertir les résumés production en DataFrame
prod_rows = []
for prod_key, prod_data in summaries_prod_raw.items():
    article_id = prod_data.get("article_id")
    
    # Extraire les stratégies disponibles
    strategies = prod_data.get("strategies", {})
    for strategy_name, strategy_data in strategies.items():
        prod_rows.append({
            "prod_key": prod_key,
            "article_id": article_id,
            "strategy": strategy_name,
            "summary_text": strategy_data.get("summary", ""),
            "coherence": strategy_data.get("metrics", {}).get("coherence"),
            "factuality": strategy_data.get("metrics", {}).get("factuality"),
            "composite_score": strategy_data.get("metrics", {}).get("composite_score")
        })

df_prod_expanded = pd.DataFrame(prod_rows)
print(f"Résumés production expandés: {len(df_prod_expanded)} entrées")
print(f"Stratégies trouvées: {df_prod_expanded['strategy'].unique()}")

# Joindre avec les articles pour récupérer les source_id
df_articles_to_prod = df_prod_expanded.merge(
    df_articles[["id", "source_id", "title", "url"]], 
    left_on="article_id", 
    right_on="id", 
    how="left"
)

# Statistiques du mapping
mapped_articles = df_articles_to_prod["source_id"].notna().sum()
print(f"Articles mappés avec succès: {mapped_articles}/{len(df_articles_to_prod)} ({mapped_articles/len(df_articles_to_prod)*100:.1f}%)")

# Créer la table de mapping Articles → Production
mapping_articles_prod = df_articles_to_prod[[
    "prod_key", "article_id", "strategy", "source_id", "title"
]].copy()

print("\nAperçu mapping Articles → Production:")
print(mapping_articles_prod.head())

=== MAPPING ARTICLES → RÉSUMÉS PRODUCTION ===
Résumés production expandés: 372 entrées
Stratégies trouvées: ['adaptive' 'confidence_weighted']
Articles mappés avec succès: 372/372 (100.0%)

Aperçu mapping Articles → Production:
  prod_key  article_id             strategy         source_id  \
0        0           0             adaptive  0c862e6732c5a41a   
1        0           0  confidence_weighted  0c862e6732c5a41a   
2        1           1             adaptive  89580b321df35a6a   
3        1           1  confidence_weighted  89580b321df35a6a   
4        2           2             adaptive  36e1a50865a65b52   

                                               title  
0                           Article synthétique ID 0  
1                           Article synthétique ID 0  
2  Almost a third of people in Gaza not eating fo...  
3  Almost a third of people in Gaza not eating fo...  
4  'I witnessed war crimes' in Gaza, former worke...  


In [6]:
# =========================
# 5) Mapping Résumés Production → Level1
# =========================

print("=== MAPPING RÉSUMÉS PRODUCTION → LEVEL1 ===")

# Extraire les informations des IDs Level1
df_level1_expanded = df_level1.copy()
df_level1_expanded[["base_id", "strategy"]] = df_level1_expanded["id"].apply(
    lambda x: pd.Series(extract_strategy_and_id(x))
)

# Convertir base_id en numérique pour correspondance
df_level1_expanded["base_id_num"] = pd.to_numeric(df_level1_expanded["base_id"], errors="coerce")

print(f"Level1 IDs analysés: {len(df_level1_expanded)}")
print(f"Stratégies Level1: {df_level1_expanded['strategy'].value_counts().to_dict()}")

# CORRECTION: Joindre Production avec Level1 via base_id et stratégie
# Utiliser suffixes pour éviter les conflits de colonnes
mapping_prod_level1 = df_articles_to_prod.merge(
    df_level1_expanded,
    left_on=["prod_key", "strategy"],
    right_on=["base_id", "strategy"],
    how="inner",
    suffixes=("_prod", "_level1")  # Éviter les conflits de colonnes
)

print(f"Correspondances Production → Level1: {len(mapping_prod_level1)}")

# Vérifier la couverture
level1_matched = len(mapping_prod_level1)
level1_total = len(df_level1)
print(f"Couverture Level1: {level1_matched}/{level1_total} ({level1_matched/level1_total*100:.1f}%)")

if level1_matched < level1_total:
    unmatched_level1 = df_level1[~df_level1["id"].isin(mapping_prod_level1["id_level1"])]
    print(f"⚠️  {len(unmatched_level1)} entrées Level1 non mappées")
    print("Échantillon non mappé:", unmatched_level1["id"].head().tolist())

print("\\nAperçu mapping Production → Level1:")
# Utiliser id_level1 au lieu de id pour éviter l'ambiguïté
print(mapping_prod_level1[["prod_key", "strategy", "id_level1", "source_id"]].head())

=== MAPPING RÉSUMÉS PRODUCTION → LEVEL1 ===
Level1 IDs analysés: 372
Stratégies Level1: {'adaptive': 186, 'confidence_weighted': 186}
Correspondances Production → Level1: 372
Couverture Level1: 372/372 (100.0%)
\nAperçu mapping Production → Level1:
  prod_key             strategy              id_level1         source_id
0        0             adaptive             0_adaptive  0c862e6732c5a41a
1        0  confidence_weighted  0_confidence_weighted  0c862e6732c5a41a
2        1             adaptive             1_adaptive  89580b321df35a6a
3        1  confidence_weighted  1_confidence_weighted  89580b321df35a6a
4        2             adaptive             2_adaptive  36e1a50865a65b52


In [7]:
# =========================
# 6) Mapping Level1 → Level2 (Direct)
# =========================

print("=== MAPPING LEVEL1 → LEVEL2 ===")

# Level2 utilise summary_id qui correspond aux IDs Level1
mapping_level1_level2 = df_level2.merge(
    df_level1[["id"]],
    left_on="summary_id",
    right_on="id",
    how="inner"
)

print(f"Correspondances Level1 → Level2: {len(mapping_level1_level2)}")
print(f"Couverture Level2: {len(mapping_level1_level2)}/{len(df_level2)} ({len(mapping_level1_level2)/len(df_level2)*100:.1f}%)")

# Vérifier l'intégrité
if len(mapping_level1_level2) == len(df_level2) == len(df_level1):
    print(" Mapping Level1 → Level2 parfait (1:1)")
else:
    print("  Incohérence dans le mapping Level1 → Level2")

print("\nAperçu mapping Level1 → Level2:")
print(mapping_level1_level2[["summary_id", "tier", "is_valid"]].head())

=== MAPPING LEVEL1 → LEVEL2 ===
Correspondances Level1 → Level2: 372
Couverture Level2: 372/372 (100.0%)
 Mapping Level1 → Level2 parfait (1:1)

Aperçu mapping Level1 → Level2:
   summary_id       tier  is_valid
0  0_adaptive       GOOD      True
1  1_adaptive       GOOD      True
2  2_adaptive  EXCELLENT      True
3  3_adaptive  EXCELLENT      True
4  4_adaptive       GOOD      True


In [8]:
# =========================
# 7) Création de la table de mapping unifiée
# =========================

print("=== CRÉATION DE LA TABLE DE MAPPING UNIFIÉE ===")

# CORRECTION: Joindre toutes les informations en une seule table
# Utiliser id_level1 au lieu de id pour éviter l'ambiguïté
unified_mapping = mapping_prod_level1.merge(
    df_level2[["summary_id", "tier", "is_valid", "level3_priority_final"]],
    left_on="id_level1",
    right_on="summary_id",
    how="left"
)

print("Colonnes disponibles après merge:")
print(sorted(unified_mapping.columns.tolist()))

# CORRECTION: Sélectionner les colonnes importantes pour le mapping final
# Utiliser les noms de colonnes RÉELS après le merge avec suffixes
unified_mapping_clean = unified_mapping[[
    # IDs de liaison
    "article_id", "prod_key", "id_level1", "summary_id",
    # Informations source
    "source_id", "title", "url", "strategy",
    # Métriques Level1
    "original_grade", "coherence_level1", "factuality_level1", "is_suspect",
    # Métriques Level2
    "tier", "is_valid", "level3_priority_final"
]].copy()

# Renommer pour clarté
unified_mapping_clean = unified_mapping_clean.rename(columns={
    "id_level1": "level1_id",
    "summary_id": "level2_id",
    "coherence_level1": "coherence",
    "factuality_level1": "factuality"
})

print(f"Table de mapping unifiée créée: {len(unified_mapping_clean)} entrées")
print(f"Colonnes finales: {unified_mapping_clean.columns.tolist()}")

# Statistiques de couverture finale
total_entries = len(unified_mapping_clean)
with_source_id = unified_mapping_clean["source_id"].notna().sum()
coverage_pct = with_source_id / total_entries * 100 if total_entries > 0 else 0

print(f"\\n=== COUVERTURE FINALE ===")
print(f"Entrées avec source_id: {with_source_id}/{total_entries} ({coverage_pct:.1f}%)")
print(f"Articles uniques mappés: {unified_mapping_clean['article_id'].nunique()}")
print(f"Stratégies mappées: {unified_mapping_clean['strategy'].value_counts().to_dict()}")

print("\\nAperçu mapping unifié:")
print(unified_mapping_clean.head())

=== CRÉATION DE LA TABLE DE MAPPING UNIFIÉE ===
Colonnes disponibles après merge:
['article_id', 'base_id', 'base_id_num', 'coherence_level1', 'coherence_prod', 'composite_score', 'confidence_score', 'entities_detected', 'fact_check_candidates_count', 'factuality_level1', 'factuality_prod', 'id_level1', 'id_prod', 'is_suspect', 'is_valid', 'level0_status', 'level3_priority_final', 'num_issues', 'original_grade', 'priority_score', 'processing_time_ms', 'prod_key', 'risk_level', 'source_id', 'strategy', 'summary_id', 'summary_text', 'suspicious_entities', 'tier', 'title', 'url', 'word_count']
Table de mapping unifiée créée: 372 entrées
Colonnes finales: ['article_id', 'prod_key', 'level1_id', 'level2_id', 'source_id', 'title', 'url', 'strategy', 'original_grade', 'coherence', 'factuality', 'is_suspect', 'tier', 'is_valid', 'level3_priority_final']
\n=== COUVERTURE FINALE ===
Entrées avec source_id: 372/372 (100.0%)
Articles uniques mappés: 186
Stratégies mappées: {'adaptive': 186, 'confi

In [9]:
# =========================
# 8) Export des tables de mapping
# =========================

print("=== EXPORT DES TABLES DE MAPPING ===")

# 8.1) Table de mapping unifiée principale
unified_file = out_dir / "unified_mapping_complete.csv"
unified_mapping_clean.to_csv(unified_file, index=False, encoding="utf-8")
print(f"[OK] Mapping unifie exporte: {unified_file}")

# 8.2) Table simplifiée pour Level2/Level3 (summary_id → source_id)
level2_mapping = unified_mapping_clean[
    ["level1_id", "level2_id", "source_id", "article_id", "strategy", "title"]
].drop_duplicates()

level2_file = out_dir / "level2_source_mapping.csv"
level2_mapping.to_csv(level2_file, index=False, encoding="utf-8")
print(f"[OK] Mapping Level2 exporte: {level2_file}")

# 8.3) Table source_id → métadonnées articles (CORRIGÉ)
source_metadata = df_articles[[
    "id", "source_id", "title", "url", "source", "published", "text"
]].copy()

metadata_file = out_dir / "source_metadata.csv"
source_metadata.to_csv(metadata_file, index=False, encoding="utf-8")
print(f"[OK] Metadonnees sources exportees: {metadata_file}")

# 8.4) Statistiques de mapping pour Level3 (CORRIGÉ: conversion types)
mapping_stats = {
    "total_articles": int(len(df_articles)),  # Conversion explicite en int Python
    "articles_used_in_production": int(unified_mapping_clean["article_id"].nunique()),
    "total_level1_entries": int(len(df_level1)),
    "total_level2_entries": int(len(df_level2)),
    "entries_with_source_id": int(with_source_id),
    "coverage_percentage": float(coverage_pct),  # Conversion en float Python
    "strategies": {k: int(v) for k, v in unified_mapping_clean["strategy"].value_counts().to_dict().items()},  # Conversion des valeurs
    "synthetic_articles_created": 1,  # Article ID 0 synthétique
    "mapping_created_at": datetime.now().isoformat()
}

stats_file = out_dir / "mapping_statistics.json"
with open(stats_file, "w", encoding="utf-8") as f:
    json.dump(mapping_stats, f, indent=2, ensure_ascii=False)
print(f"[OK] Statistiques exportees: {stats_file}")

# 8.5) NOUVEAU: Mise à jour du fichier Level2 avec les source_id
print("\\n=== MISE A JOUR LEVEL2 AVEC SOURCE_ID ===")

# Charger le fichier Level2 existant
level2_original_file = out_dir / "level2_simplified_results_with_ids.csv"
df_level2_original = pd.read_csv(level2_original_file)

# Créer le mapping summary_id → source_id
source_id_mapping = unified_mapping_clean[["level2_id", "source_id"]].drop_duplicates()
source_id_mapping = source_id_mapping.rename(columns={"level2_id": "summary_id"})

# Fusionner avec Level2 pour remplir les source_id
df_level2_updated = df_level2_original.drop(columns=["source_id"], errors="ignore").merge(
    source_id_mapping, on="summary_id", how="left"
)

# Sauvegarder le fichier Level2 mis à jour
level2_updated_file = out_dir / "level2_simplified_results_with_ids.csv"
df_level2_updated.to_csv(level2_updated_file, index=False, encoding="utf-8")
print(f"[OK] Level2 mis a jour avec source_id: {level2_updated_file}")

# Statistiques de la mise à jour
updated_source_ids = df_level2_updated["source_id"].notna().sum()
update_coverage = updated_source_ids / len(df_level2_updated) * 100
print(f"   Source_ID ajoutes: {updated_source_ids}/{len(df_level2_updated)} ({update_coverage:.1f}%)")

print("\\n=== EXPORTS TERMINES ===")
print(f"Fichiers crees dans: {out_dir}")
print("- unified_mapping_complete.csv (mapping complet)")
print("- level2_source_mapping.csv (pour Level2/Level3)")
print("- source_metadata.csv (metadonnees articles)")
print("- mapping_statistics.json (statistiques)")
print("- level2_simplified_results_with_ids.csv (mis a jour avec source_id)")

# 8.6) MISE À JOUR des autres fichiers Level2 également
print("\\n=== MISE A JOUR AUTRES FICHIERS LEVEL2 ===")

# Mise à jour du fichier prioritaire
priority_file = out_dir / "level2_simplified_priority_cases_with_ids.csv"
try:
    if priority_file.exists():
        df_priority_original = pd.read_csv(priority_file)
        df_priority_updated = df_priority_original.drop(columns=["source_id"], errors="ignore").merge(
            source_id_mapping, on="summary_id", how="left"
        )
        df_priority_updated.to_csv(priority_file, index=False, encoding="utf-8")
        print(f"[OK] Fichier prioritaire mis a jour: {priority_file}")
except Exception as e:
    print(f"[ATTENTION] Erreur mise a jour fichier prioritaire: {e}")

# Mise à jour du JSON Level2
json_file = out_dir / "level2_output_with_source_id.json"
try:
    if json_file.exists():
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)
        
        # Mettre à jour les résultats dans le JSON
        if "results" in json_data:
            source_mapping_dict = dict(zip(source_id_mapping["summary_id"], source_id_mapping["source_id"]))
            for result in json_data["results"]:
                summary_id = result.get("summary_id")
                if summary_id in source_mapping_dict:
                    result["source_id"] = source_mapping_dict[summary_id]
        
        # Sauvegarder le JSON mis à jour
        with open(json_file, "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        print(f"[OK] JSON Level2 mis a jour: {json_file}")
except Exception as e:
    print(f"[ATTENTION] Erreur mise a jour JSON: {e}")

print("\\n=== MAPPING UNIFIE TERMINE AVEC SUCCES ===")
print(f"Coverage finale: {update_coverage:.1f}%")
print("Tous les fichiers Level2 ont ete mis a jour avec les source_id")

=== EXPORT DES TABLES DE MAPPING ===
[OK] Mapping unifie exporte: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\unified_mapping_complete.csv
[OK] Mapping Level2 exporte: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\level2_source_mapping.csv
[OK] Metadonnees sources exportees: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\source_metadata.csv
[OK] Statistiques exportees: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\mapping_statistics.json
\n=== MISE A JOUR LEVEL2 AVEC SOURCE_ID ===
[OK] Level2 mis a jour avec source_id: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\outputs\level2_simplified_results_with_ids.csv
   Source_ID ajoutes: 372/372 (100.0%)
\n=== EXPORTS TERMINES ===
Fichiers crees dans: c:\Users\beedi.goua_square-ma\Desktop\Gheb\proj

In [10]:
# =========================
# 9) Validation et contrôles qualité
# =========================

print("=== VALIDATION ET CONTRÔLES QUALITÉ ===")

# 9.1) Vérifier l'intégrité des mappings
print("1. Vérification de l'intégrité:")

# Unicité des IDs
duplicate_level1 = unified_mapping_clean["level1_id"].duplicated().sum()
duplicate_level2 = unified_mapping_clean["level2_id"].duplicated().sum()
print(f"   - Doublons Level1 IDs: {duplicate_level1}")
print(f"   - Doublons Level2 IDs: {duplicate_level2}")

# 9.2) Analyser la répartition par stratégie
print("\n2. Répartition par stratégie:")
strategy_stats = unified_mapping_clean.groupby("strategy").agg({
    "source_id": "count",
    "is_valid": "sum",
    "level3_priority_final": "mean"
}).round(3)
strategy_stats.columns = ["total_entries", "valid_entries", "avg_priority"]
print(strategy_stats)

# 9.3) Analyser la couverture par grade
print("\n3. Couverture par grade original:")
grade_coverage = unified_mapping_clean.groupby("original_grade").agg({
    "source_id": ["count", lambda x: x.notna().sum()]
})
grade_coverage.columns = ["total", "with_source_id"]
grade_coverage["coverage_pct"] = (grade_coverage["with_source_id"] / grade_coverage["total"] * 100).round(1)
print(grade_coverage)

# 9.4) Identifier les cas problématiques
print("\n4. Cas problématiques:")
missing_source_id = unified_mapping_clean[unified_mapping_clean["source_id"].isna()]
print(f"   - Entrées sans source_id: {len(missing_source_id)}")

if len(missing_source_id) > 0:
    print("   Échantillon sans source_id:")
    print(missing_source_id[["level1_id", "article_id", "strategy", "original_grade"]].head())

# 9.5) Résumé final de validation
print(f"\n=== RÉSUMÉ DE VALIDATION ===")
print(f" Articles sources: {len(df_articles)}")
print(f" Résumés production: {len(summaries_prod_raw)} → {len(df_prod_expanded)} (avec stratégies)")
print(f" Level1 mappé: {len(unified_mapping_clean)}/{len(df_level1)} ({len(unified_mapping_clean)/len(df_level1)*100:.1f}%)")
print(f" Level2 mappé: {len(unified_mapping_clean)}/{len(df_level2)} ({len(unified_mapping_clean)/len(df_level2)*100:.1f}%)")
print(f" Source_ID coverage: {coverage_pct:.1f}%")

# Recommandations
print(f"\n=== RECOMMANDATIONS ===")
if coverage_pct >= 95:
    print(" Excellent: Couverture source_id >95% - Level3 peut fonctionner pleinement")
elif coverage_pct >= 80:
    print(" Bon: Couverture source_id >80% - Level3 fonctionnel avec limitations mineures")
elif coverage_pct >= 60:
    print("  Moyen: Couverture source_id >60% - Level3 fonctionnel mais avec limitations")
else:
    print(" Faible: Couverture source_id <60% - Améliorer le mapping avant Level3")

print("\n MAPPING UNIFIÉ TERMINÉ AVEC SUCCÈS")

=== VALIDATION ET CONTRÔLES QUALITÉ ===
1. Vérification de l'intégrité:
   - Doublons Level1 IDs: 0
   - Doublons Level2 IDs: 0

2. Répartition par stratégie:
                     total_entries  valid_entries  avg_priority
strategy                                                       
adaptive                       186            159         0.734
confidence_weighted            186              0         0.829

3. Couverture par grade original:
                total  with_source_id  coverage_pct
original_grade                                     
A                  62              62         100.0
A+                 60              60         100.0
B                  11              11         100.0
B+                158             158         100.0
C                  17              17         100.0
D                  64              64         100.0

4. Cas problématiques:
   - Entrées sans source_id: 0

=== RÉSUMÉ DE VALIDATION ===
 Articles sources: 587
 Résumés production: 186 →